# Stock Trades by Members of the US House of Representatives

This project uses public data about the stock trades made by members of the US House of Representatives. This data is collected and maintained by Timothy Carambat as part of the [House Stock Watcher](https://housestockwatcher.com/) project. The project describes itself as follows:

> With recent and ongoing investigations of incumbent congressional members being investigated for potentially violating the STOCK act. This website compiles this publicly available information in a format that is easier to digest then the original PDF source.
>
> Members of Congress must report periodic reports of their asset transactions. This website is purely for an informative purpose and aid in transparency.
>
> This site does not manipluate or censor any of the information from the original source. All data is transcribed by our community of contributors, which you can join for free by going to our transcription tool. Our moderation team takes great care in ensuring the accuracy of the information.
>
> This site is built and maintained by Timothy Carambat and supported with our contributors.

Some interesting questions to consider for this data set include:

- Is there a difference in stock trading behavior between political parties? For example:
    - does one party trade more often?
    - does one party make larger trades?
    - do the two parties invest in different stocks or sectors? For instance, do Democrats invest in Tesla more than Republicans?
- What congresspeople have made the most trades?
- What companies are most traded by congresspeople?
- Is there evidence of insider trading? For example, Boeing stock dropped sharply in February 2020. Were there a suspiciously-high number of sales of Boeing before the drop?
- When are stocks bought and sold? Is there a day of the week that is most common? Or a month of the year?

### Getting the Data

The full data set of stock trade disclosures is available as a CSV or as JSON at https://housestockwatcher.com/api.

This data set does not, however, contain the political affiliation of the congresspeople. If you wish to investigate a question that relies on having this information, you'll need to find another dataset that contains it and perform a merge. *Hint*: Kaggle is a useful source of data sets.


### Cleaning and EDA

- Clean the data.
    - Certain fields have "missing" data that isn't labeled as missing. For example, there are fields with the value "--." Do some exploration to find those values and convert them to null values.
    - You may also want to clean up the date columns to enable time-series exploration.
- Understand the data in ways relevant to your question using univariate and bivariate analysis of the data as well as aggregations.


### Assessment of Missingness

- Assess the missingness per the requirements in `project03.ipynb`

### Hypothesis Test / Permutation Test
Find a hypothesis test or permutation test to perform. You can use the questions at the top of the notebook for inspiration.

# Summary of Findings

### Introduction
TODO

### Cleaning and EDA
TODO

### Assessment of Missingness
TODO

### Hypothesis Test
TODO

# Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import missingno as msno
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Cleaning and EDA

In [2]:
fp = os.path.join('data/', 'all_transactions.csv')
all_transactions = pd.read_csv(fp)
all_transactions.isna().sum()

disclosure_year              0
disclosure_date              0
transaction_date             0
owner                     4529
ticker                       0
asset_description            4
type                         0
amount                       0
representative               0
district                     0
ptr_link                     0
cap_gains_over_200_usd       0
dtype: int64

So let us begin by finding all potential fake missingness and turning it into real missingness. It appears, after some rigorous analysis (looking at each series potential values), that the only fake missingness exists in the 'owner' and 'ticker' columns. We will now replace those values with NaNs. In summary:

Missingness in data:
- Owner column has fake missing ('--') and real missing
- Ticker column has fake missing ('--')
- Asset description column has real missing but only 4 so we can hand clean this

In [3]:
all_transactions.iloc[732] #Replace NaN in a_d with Ball Corporation
all_transactions.iloc[3262] #Replace NaN in a_d with CELO
all_transactions.iloc[10537] #Replace NaN in a_d with URGO
all_transactions.iloc[10538] #Replace NaN in a_d with URGO

disclosure_year                                                        2021
disclosure_date                                                  03/18/2021
transaction_date                                                 2021-02-18
owner                                                                   NaN
ticker                                                                 URGO
asset_description                                                       NaN
type                                                               purchase
amount                                                     $1,001 - $15,000
representative                                              Hon. Brian Mast
district                                                               FL18
ptr_link                  https://disclosures-clerk.house.gov/public_dis...
cap_gains_over_200_usd                                                False
Name: 10538, dtype: object

In [4]:
all_transactions.at[732, 'asset_description'] = 'Ball Corporation'
all_transactions.at[3262, 'asset_description'] = 'CELO'
all_transactions.at[10537, 'asset_description'] = 'URGO'
all_transactions.at[10538, 'asset_description'] = 'URGO'

Our current strategy for tickers is to ignore the ticker and instead use the asset description wherever we can. The ticker is currently hypothesized to be NMAR after some brief exploration of some missing tickers: Missing dependent upon if the asset is listed on the NASDAQ. This can be further proven by understanding that the collection process populates the form according to NASDAQ tickers. 

In the following cell we sorted the transaction date values to find the smallest values from a string perspective, and discovered several anomalies. For these anomalies our intention is to hand clean the data and make sure that the date listed matches the date reported in the PDF sent in to Congress. 

In [5]:
all_transactions['transaction_date'].sort_values().head(10)

2069    0009-06-09
9381    0021-06-22
3496    0021-08-02
9382    0201-06-22
8662    2012-06-19
3489    2018-09-08
3488    2018-09-09
9627    2018-12-27
7875    2019-01-09
7535    2019-01-09
Name: transaction_date, dtype: object

In [6]:
all_transactions.iloc[8662] #Correct transaction date is: 06/19/2020
all_transactions.iloc[2069] #Correct transaction date is: 06/09/2021
all_transactions.iloc[9381] #Correct transaction date is: 06/22/2021
all_transactions.iloc[3496] #Correct transaction date is: 08/02/2021
all_transactions.iloc[9382] #Correct transaction date is: 06/22/2021

disclosure_year                                                        2021
disclosure_date                                                  07/16/2021
transaction_date                                                 0201-06-22
owner                                                                   NaN
ticker                                                                   KR
asset_description                                            Kroger Company
type                                                           sale_partial
amount                                                     $1,001 - $15,000
representative                                       Hon. James E Hon Banks
district                                                               IN03
ptr_link                  https://disclosures-clerk.house.gov/public_dis...
cap_gains_over_200_usd                                                False
Name: 9382, dtype: object

In [7]:
all_transactions.at[8662, 'transaction_date'] = '2020-06-19'
all_transactions.at[2069, 'transaction_date'] = '2021-06-09'
all_transactions.at[9381, 'transaction_date'] = '2021-06-22'
all_transactions.at[3496, 'transaction_date'] = '2021-08-02'
all_transactions.at[9382, 'transaction_date'] = '2021-06-22'

In [8]:
all_transactions['transaction_date'].sort_values().head(10)

3489    2018-09-08
3488    2018-09-09
9627    2018-12-27
7475    2019-01-09
7535    2019-01-09
7797    2019-01-09
7469    2019-01-09
7560    2019-01-09
7420    2019-01-09
7570    2019-01-09
Name: transaction_date, dtype: object

In [9]:
transactions_w_nan = all_transactions.replace('--', np.nan)

In [10]:
transactions_w_nan.isna().sum()

disclosure_year              0
disclosure_date              0
transaction_date             0
owner                     5844
ticker                    1023
asset_description            0
type                         0
amount                       0
representative               0
district                     0
ptr_link                     0
cap_gains_over_200_usd       0
dtype: int64

In [11]:
transactions_w_nan['transaction_date'] = pd.to_datetime(transactions_w_nan['transaction_date'])
transactions_w_nan['transaction_date'].dtypes

dtype('<M8[ns]')

In [12]:
transactions_w_nan['disclosure_date'] = pd.to_datetime(transactions_w_nan['disclosure_date'])
transactions_w_nan['disclosure_date'].dtypes

dtype('<M8[ns]')

Using some highly advanced ctrl+c, ctrl+v technology, we obtained a dataset of names and parties from the official house website. After viewing these names and exploring the names in our dataset in depth, we realized how messy our data really was. So we're going to replace the names of our messy data with the names from our clean data, and update the parties once this is done.

In [13]:
parties_fp = os.path.join('data/', 'names_and_parties.csv')
parties = pd.read_csv(parties_fp)
parties.head()

,REPRESENTATIVE,PARTY,STATE,DISTRICT
0,"Adams, Alma S.",Democrat,North Carolina (NC),12th
1,"Aderholt, Robert B.",Republican,Alabama (AL),4th
2,"Aguilar, Pete",Democrat,California (CA),31st
3,"Allen, Rick W.",Republican,Georgia (GA),12th
4,"Allred, Colin Z.",Democrat,Texas (TX),32nd


In [44]:
parties['DISTRICT'] = parties['DISTRICT'].mask(parties['DISTRICT'].str.len() > 4, other='00')
dist_code = (parties['STATE'].str.slice(-3, -1)) + parties['DISTRICT'].str.slice(0, -2).str.zfill(2)
w_code = parties.assign(code=dist_code)
names_and_codes = w_code.set_index('code')['REPRESENTATIVE']

In [45]:
dist_rep_pairs = dict(zip(names_and_codes.index, names_and_codes.values))

In [46]:
w_party = transactions_w_nan.copy()
w_party["representative"] = transactions_w_nan["district"].replace(dist_rep_pairs)

Interestingly enough we seem to be missing three different representatives/districts from our new cleaned data. After some examination these appear to be two at large/not numbered representatives and one representative who is currently retired. Since we've only missed three, we can go in by hand and replace these values ourselves.

In [ ]:
reps_by_party = parties[['REPRESENTATIVE', 'PARTY']].set_index('REPRESENTATIVE')
w_party = w_party.set_index('representative').merge(reps_by_party, how='left', left_index=True, right_index=True)
w_party

In [59]:
w_party.reset_index()[w_party.reset_index()['index'].str.contains('Rosendale')]

,index,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,district,ptr_link,cap_gains_over_200_usd,PARTY
10223,"Rosendale, Matthew M. Sr.",2020,2020-11-09,2020-10-13,joint,AMED,Amedisys Inc,sale_partial,"$1,001 - $15,000",MT00,https://disclosures-clerk.house.gov/public_dis...,True,Republican
10224,"Rosendale, Matthew M. Sr.",2020,2020-11-09,2020-10-13,joint,ANSS,"ANSYS, Inc.",sale_partial,"$15,001 - $50,000",MT00,https://disclosures-clerk.house.gov/public_dis...,True,Republican
10225,"Rosendale, Matthew M. Sr.",2020,2020-11-09,2020-10-13,joint,TECH,Bio-Techne Corp,sale_partial,"$1,001 - $15,000",MT00,https://disclosures-clerk.house.gov/public_dis...,True,Republican
10226,"Rosendale, Matthew M. Sr.",2020,2020-11-09,2020-10-13,joint,CHE,Chemed Corp,sale_partial,"$1,001 - $15,000",MT00,https://disclosures-clerk.house.gov/public_dis...,True,Republican
10227,"Rosendale, Matthew M. Sr.",2020,2020-11-09,2020-10-13,joint,CGNX,Cognex Corporation,sale_partial,"$1,001 - $15,000",MT00,https://disclosures-clerk.house.gov/public_dis...,True,Republican
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10715,"Rosendale, Matthew M. Sr.",2020,2020-12-23,2020-12-15,joint,OSW,OneSpaWorld Holdings Limited - Common Shares,purchase,"$100,001 - $250,000",MT00,https://disclosures-clerk.house.gov/public_dis...,False,Republican
10716,"Rosendale, Matthew M. Sr.",2020,2020-12-23,2020-12-15,joint,THR,"Thermon Group Holdings, Inc.",purchase,"$100,001 - $250,000",MT00,https://disclosures-clerk.house.gov/public_dis...,False,Republican
10717,"Rosendale, Matthew M. Sr.",2020,2020-12-23,2020-12-18,joint,VSAT,"ViaSat, Inc.",purchase,"$100,001 - $250,000",MT00,https://disclosures-clerk.house.gov/public_dis...,False,Republican
10718,"Rosendale, Matthew M. Sr.",2020,2020-12-23,2020-12-01,joint,WIX,Wix.com Ltd. - Ordinary Shares,purchase,"$1,001 - $15,000",MT00,https://disclosures-clerk.house.gov/public_dis...,False,Republican


In [50]:
transactions_w_nan[transactions_w_nan["district"] == "DC00"]['representative'].unique()

array(['Hon. Eleanor Holmes Norton'], dtype=object)

In [51]:
transactions_w_nan[transactions_w_nan["district"] == "MT00"]['representative'].unique()

array(['Hon. Greg Gianforte'], dtype=object)

In [52]:
transactions_w_nan[transactions_w_nan["district"] == "VT00"]['representative'].unique()

array(['Hon. Peter Welch'], dtype=object)

### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Test / Permutation Test

In [ ]:
# TODO